<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/Advanced-RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><div align="center"> Advanced RVC Inference:

<big> for quicker and effortless model downloads

---

[Support](https://discord.gg/hvmsukmBHE) — [GitHub](https://github.com/ArkanDash/Advanced-RVC-Inference.git)

In [ ]:

#@title ## **Setup and Installation**
import os
import subprocess
import codecs

def run_command(command, error_message, capture_output=True):
    """Runs a shell command and handles errors."""
    try:
        result = subprocess.run(command, shell=True, text=True, capture_output=capture_output, check=True)
        if capture_output:
            return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(f"❌ {error_message}: {e.stderr.strip() if e.stderr else 'No details available'}")
        raise

def main():
    print("🚀: Initializing setup...")

    # Decode repository URL
    repo_url = codecs.decode("uggcf://tvguho.pbz/NexnaQnfu/Nqinaprq-EIP-Vasrerapr.tvg", "rot_13")

    # Clone repository
    print("📥: Cloning repository...")
    run_command(f"git clone {repo_url} main_program", "Failed to clone repository")

    # Change directory
    os.chdir("main_program")

    # Install UV package manager
    print("📦: Installing UV package manager...")
    run_command("pip install uv", "Failed to install UV")

    # Create and activate virtual environment
    print("🔧: Setting up virtual environment...")
    run_command("uv venv .venv", "Failed to create virtual environment")

    # Install dependencies
    print("📌: Installing dependencies...")
    run_command("uv pip install --no-deps -r requirements.txt", "Failed to install dependencies")

    # Install PyTorch with CUDA support
    print("🔥: Installing PyTorch (CUDA 12.1)...")
    run_command("uv pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --upgrade --index-url https://download.pytorch.org/whl/cu121",
                "Failed to install PyTorch")

    # Ensure correct NumPy version
    print("🔢: Installing NumPy 1.23.5...")
    run_command("uv pip install numpy==1.23.5", "Failed to install NumPy")

    # Install pyngrok for tunneling
    print("🌐: Installing pyngrok...")
    run_command("pip install pyngrok", "Failed to install pyngrok")

    # Run prerequisite download script
    print("🔄: Downloading prerequisites...")
    run_command("source .venv/bin/activate; python programs/applio_code/rvc/lib/tools/prerequisites_download.py",
                "Failed to execute prerequisites script")

    print("✅: Setup complete! All requirements installed successfully.")

if __name__ == "__main__":
    main()

In [ ]:
#@title ## **Start**
import codecs

!git pull -q

runtime=codecs.decode("ncc.cl --funer", "rot_13")
!source .venv/bin/activate; python $runtime